In [ ]:
# yfinance not real time data causing issue 

In [8]:
import numpy as np
import pandas as pd
import time
import yfinance as yf
import plotly.graph_objs as go

# Function to place a trade (Buy or Sell)
def place_trade(order_type, lot_size, price):
    # Placeholder function to simulate placing a trade
    # Replace this with actual code to place a real trade using a trading platform or API
    print(f"Placing a {order_type} order for {lot_size} lots at price {price}")

stock_name = 'ADANIENT.BO'
lot_size = 1  # Initial lot size
profit_factor = 2  # Factor to multiply lot size in case of a loss

# Wait until the start of the next minute
current_time = pd.Timestamp.now().time()
seconds_to_next_minute = 60 - current_time.second
time.sleep(seconds_to_next_minute)

while True:
    # Interval required 1 minute
    data = yf.download(tickers=stock_name, period='1d', interval='1m')
    print(data.iloc[-1])

    # Assuming 'data' contains the historical stock data fetched using yfinance
    previous_candle = data.iloc[-1]  # Get the row for the previous candle

    if previous_candle['Close'] > previous_candle['Open']:
        print("Previous candle was bullish.")
        pre_bull_or_bear = 'bull'
        # If bullish, buy 1 lot / stock at LIMIT with price as Close price of previous candle.
        place_trade("Buy", lot_size, previous_candle['Close'])
    else:
        print("Previous candle was bearish.")
        pre_bull_or_bear = 'bear'
        # If bearish Sellshort 1 lot / stock at LIMIT with price as close price of previous candle
        place_trade("Sell", lot_size, previous_candle['Close'])

    # Wait until the end of the current minute (59th second of the minute)
    current_time = pd.Timestamp.now().time()
    seconds_to_next_minute = 57 - current_time.second
    time.sleep(seconds_to_next_minute)
    # IMP cancel all pending orders
    data = yf.download(tickers=stock_name, period='1d', interval='1m')
    print(data.iloc[-1])
    current_candle = data.iloc[-1]

    if pre_bull_or_bear == 'bull':
        # At the end of the candle, check if the position is profitable
        if previous_candle['Close'] < current_candle['Close']:
            # The position was profitable, keep the lot size at 1
            print("Position was profitable. Keeping lot size at 1.")
            lot_size = 1
        else:
            # The position was a loss, multiply lot size by profit_factor
            print("Position was a loss. Multiplying lot size by 2.")
            lot_size *= profit_factor
    else:
        # At the end of the candle, check if the position is profitable
        if previous_candle['Close'] > current_candle['Close']:
            # The position was profitable, keep the lot size at 1
            print("Position was profitable. Keeping lot size at 1.")
            lot_size = 1
        else:
            # The position was a loss, multiply lot size by profit_factor
            print("Position was a loss. Multiplying lot size by 2.")
            lot_size *= profit_factor